あらゆる情報をまずは表にして，それから可視化する

- 

# Association Rule によるルール作成

## 一時保存した実験結果の読み込み

In [2]:
import json
import os

import sys
project_dir_path = "/Users/keisukeonoue/ws/lukasiewicz_2"
sys.path.append(project_dir_path)

import pandas as pd
import numpy as np

version_nums = [151, 152, 153, 154, 155, 156, 157, 158]

# リスト内包表記を使用して複数のファイルパスを生成する
result_file_paths = [
    os.path.join(project_dir_path, f"experiment_result/tmp/version_{version_num}/result.json")
    for version_num in version_nums
]


result_dfs = []
infos = []

for path in result_file_paths:
    with open(path, 'r') as f:
        json_data = json.load(f)
    
    infos.append(json_data['rule_thr'])
    
    tmp = []

    for fold, _ in json_data['result'].items():
        result_df = pd.DataFrame(json_data['result'][fold]).T.drop(['violation_detail'], axis=1)
        tmp.append(result_df)

    result_dfs.append(tmp)

model_name_list = [
    'RuleFit Classifier (disc)', 
    'tree generator (disc)', 
    'RuleFit Classifier (conti)', 
    'tree generator (conti)', 
    'linear svm', 
    'non-linear svm', 
    'logistic regression', 
    'linear svm (L)', 
    'non-linear svm (L)', 
    'logistic regression (L)', 
    'rule_based_prediction'
]

combined_dfs = [pd.concat(dfs, ignore_index=False) for dfs in result_dfs]
dfs_mean = [combined_df.groupby(combined_df.index).mean().reindex(index=model_name_list) for combined_df in combined_dfs]
dfs_std = [combined_df.groupby(combined_df.index).std().reindex(index=model_name_list) for combined_df in combined_dfs]

for df in dfs_mean:
    display(df)

,accuracy,precision,recall,f1,auc,n_violation,n_rule,violation_rate,n_violation (instance),n_evaluation (instance),violation_rate (instance)
RuleFit Classifier (disc),0.783582,0.742431,0.521271,0.610252,0.815651,45.4,46.0,0.988235,2734.6,3148.0,0.869052
tree generator (disc),0.741791,0.627843,0.557706,0.582608,0.772322,46.0,46.0,1.0,2634.2,3148.0,0.837924
RuleFit Classifier (conti),0.732836,0.601979,0.553471,0.571908,0.795116,46.0,46.0,1.0,2632.8,3148.0,0.836848
tree generator (conti),0.78209,0.715158,0.568106,0.62746,0.8326,45.8,46.0,0.996078,2702.4,3148.0,0.85896
linear svm,0.779104,0.711436,0.551175,0.618693,0.837805,45.4,46.0,0.988235,2726.4,3148.0,0.866152
non-linear svm,0.780597,0.75907,0.479714,0.587201,0.838064,46.0,46.0,1.0,2803.4,3148.0,0.889837
logistic regression,0.78209,0.731143,0.530165,0.612464,0.840573,46.0,46.0,1.0,2759.6,3148.0,0.876497
linear svm (L),0.61791,0.471546,0.883547,0.608462,0.680065,43.0,46.0,0.926789,1408.6,3148.0,0.450271
non-linear svm (L),0.683582,0.525018,0.756815,0.607215,0.795545,44.6,46.0,0.970228,1398.2,3148.0,0.443859
logistic regression (L),0.738806,0.766883,0.29155,0.416785,0.827454,45.8,46.0,0.995833,1478.0,3148.0,0.469113


,accuracy,precision,recall,f1,auc,n_violation,n_rule,violation_rate,n_violation (instance),n_evaluation (instance),violation_rate (instance)
RuleFit Classifier (disc),0.783582,0.742431,0.521271,0.610252,0.815651,45.4,46.0,0.988235,2734.6,3148.0,0.869052
tree generator (disc),0.741791,0.627843,0.557706,0.582608,0.772322,46.0,46.0,1.0,2634.2,3148.0,0.837924
RuleFit Classifier (conti),0.732836,0.601979,0.553471,0.571908,0.795116,46.0,46.0,1.0,2632.8,3148.0,0.836848
tree generator (conti),0.78209,0.715158,0.568106,0.62746,0.8326,45.8,46.0,0.996078,2702.4,3148.0,0.85896
linear svm,0.779104,0.711436,0.551175,0.618693,0.837805,45.4,46.0,0.988235,2726.4,3148.0,0.866152
non-linear svm,0.780597,0.75907,0.479714,0.587201,0.838064,46.0,46.0,1.0,2803.4,3148.0,0.889837
logistic regression,0.78209,0.731143,0.530165,0.612464,0.840573,46.0,46.0,1.0,2759.6,3148.0,0.876497
linear svm (L),0.61791,0.471546,0.883547,0.608462,0.680065,43.0,46.0,0.926789,1408.6,3148.0,0.450271
non-linear svm (L),0.683582,0.525018,0.756815,0.607215,0.795545,44.6,46.0,0.970228,1398.2,3148.0,0.443859
logistic regression (L),0.738806,0.766883,0.29155,0.416785,0.827454,45.8,46.0,0.995833,1478.0,3148.0,0.469113


,accuracy,precision,recall,f1,auc,n_violation,n_rule,violation_rate,n_violation (instance),n_evaluation (instance),violation_rate (instance)
RuleFit Classifier (disc),0.783582,0.742431,0.521271,0.610252,0.815651,30.8,31.4,0.984615,1620.8,1918.6,0.846207
tree generator (disc),0.741791,0.627843,0.557706,0.582608,0.772322,31.4,31.4,1.0,1553.4,1918.6,0.810718
RuleFit Classifier (conti),0.732836,0.601979,0.553471,0.571908,0.795116,31.4,31.4,1.0,1557.4,1918.6,0.813508
tree generator (conti),0.78209,0.715158,0.568106,0.62746,0.8326,31.2,31.4,0.994872,1603.8,1918.6,0.836936
linear svm,0.779104,0.711436,0.551175,0.618693,0.837805,30.8,31.4,0.984615,1620.2,1918.6,0.845419
non-linear svm,0.780597,0.75907,0.479714,0.587201,0.838064,31.4,31.4,1.0,1673.8,1918.6,0.871507
logistic regression,0.78209,0.731143,0.530165,0.612464,0.840573,31.4,31.4,1.0,1643.0,1918.6,0.856911
linear svm (L),0.616418,0.467204,0.898474,0.609567,0.69472,26.6,31.4,0.835733,794.0,1918.6,0.412356
non-linear svm (L),0.708955,0.546798,0.749184,0.621242,0.809149,30.2,31.4,0.964305,827.4,1918.6,0.430133
logistic regression (L),0.732836,0.801645,0.249993,0.376407,0.832998,31.0,31.4,0.9875,940.4,1918.6,0.486252


,accuracy,precision,recall,f1,auc,n_violation,n_rule,violation_rate,n_violation (instance),n_evaluation (instance),violation_rate (instance)
RuleFit Classifier (disc),0.783582,0.742431,0.521271,0.610252,0.815651,14.6,15.2,0.971429,581.2,739.6,0.782273
tree generator (disc),0.741791,0.627843,0.557706,0.582608,0.772322,15.2,15.2,1.0,548.2,739.6,0.736835
RuleFit Classifier (conti),0.732836,0.601979,0.553471,0.571908,0.795116,15.2,15.2,1.0,550.4,739.6,0.740708
tree generator (conti),0.78209,0.715158,0.568106,0.62746,0.8326,15.0,15.2,0.990476,573.2,739.6,0.771385
linear svm,0.779104,0.711436,0.551175,0.618693,0.837805,14.6,15.2,0.971429,580.2,739.6,0.780355
non-linear svm,0.780597,0.75907,0.479714,0.587201,0.838064,15.2,15.2,1.0,607.4,739.6,0.815294
logistic regression,0.78209,0.731143,0.530165,0.612464,0.840573,15.2,15.2,1.0,592.2,739.6,0.796663
linear svm (L),0.540299,0.429574,0.868124,0.565006,0.654651,11.2,15.2,0.712711,268.6,739.6,0.354002
non-linear svm (L),0.732836,0.579031,0.697701,0.626487,0.815141,13.8,15.2,0.912711,362.8,739.6,0.490148
logistic regression (L),0.728358,0.828788,0.212169,0.335828,0.833443,14.6,15.2,0.957949,471.8,739.6,0.650522


,accuracy,precision,recall,f1,auc,n_violation,n_rule,violation_rate,n_violation (instance),n_evaluation (instance),violation_rate (instance)
RuleFit Classifier (disc),0.783582,0.742431,0.521271,0.610252,0.815651,4.2,4.8,0.925,96.8,175.8,0.527237
tree generator (disc),0.741791,0.627843,0.557706,0.582608,0.772322,4.8,4.8,1.0,88.8,175.8,0.486147
RuleFit Classifier (conti),0.732836,0.601979,0.553471,0.571908,0.795116,4.8,4.8,1.0,94.0,175.8,0.515464
tree generator (conti),0.78209,0.715158,0.568106,0.62746,0.8326,4.6,4.8,0.975,100.8,175.8,0.543868
linear svm,0.779104,0.711436,0.551175,0.618693,0.837805,4.2,4.8,0.925,98.8,175.8,0.531184
non-linear svm,0.780597,0.75907,0.479714,0.587201,0.838064,4.8,4.8,1.0,111.6,175.8,0.592676
logistic regression,0.78209,0.731143,0.530165,0.612464,0.840573,4.8,4.8,1.0,103.8,175.8,0.555287
linear svm (L),0.389552,0.330053,0.862182,0.475233,0.479372,3.0,4.8,0.533333,38.4,175.8,0.147824
non-linear svm (L),0.707463,0.537475,0.79283,0.634478,0.812482,3.4,4.8,0.708333,58.2,175.8,0.313441
logistic regression (L),0.761194,0.763833,0.389945,0.514601,0.83093,4.8,4.8,1.0,122.0,175.8,0.662662


,accuracy,precision,recall,f1,auc,n_violation,n_rule,violation_rate,n_violation (instance),n_evaluation (instance),violation_rate (instance)
RuleFit Classifier (disc),0.783582,0.742431,0.521271,0.610252,0.815651,1.2,1.8,0.85,8.6,44.6,0.156725
tree generator (disc),0.741791,0.627843,0.557706,0.582608,0.772322,1.8,1.8,1.0,10.0,44.6,0.237624
RuleFit Classifier (conti),0.732836,0.601979,0.553471,0.571908,0.795116,1.8,1.8,1.0,12.6,44.6,0.262864
tree generator (conti),0.78209,0.715158,0.568106,0.62746,0.8326,1.6,1.8,0.95,11.6,44.6,0.243154
linear svm,0.779104,0.711436,0.551175,0.618693,0.837805,1.2,1.8,0.85,9.0,44.6,0.164254
non-linear svm,0.780597,0.75907,0.479714,0.587201,0.838064,1.8,1.8,1.0,14.2,44.6,0.280367
logistic regression,0.78209,0.731143,0.530165,0.612464,0.840573,1.8,1.8,1.0,11.4,44.6,0.225741
linear svm (L),0.325373,0.325373,1.0,0.489178,0.250784,0.0,1.8,0.0,0.0,44.6,0.0
non-linear svm (L),0.720896,0.550099,0.781269,0.644552,0.821473,0.4,1.8,0.15,3.6,44.6,0.050752
logistic regression (L),0.765672,0.76142,0.419501,0.537892,0.828,1.8,1.8,1.0,16.2,44.6,0.33439


,accuracy,precision,recall,f1,auc,n_violation,n_rule,violation_rate,n_violation (instance),n_evaluation (instance),violation_rate (instance)
RuleFit Classifier (disc),0.783582,0.742431,0.521271,0.610252,0.815651,0.8,1.2,0.8,2.0,29.2,0.069582
tree generator (disc),0.741791,0.627843,0.557706,0.582608,0.772322,1.2,1.2,1.0,5.2,29.2,0.18568
RuleFit Classifier (conti),0.732836,0.601979,0.553471,0.571908,0.795116,1.2,1.2,1.0,6.2,29.2,0.213997
tree generator (conti),0.78209,0.715158,0.568106,0.62746,0.8326,1.0,1.2,0.9,5.2,29.2,0.18141
linear svm,0.779104,0.711436,0.551175,0.618693,0.837805,0.8,1.2,0.8,2.6,29.2,0.090093
non-linear svm,0.780597,0.75907,0.479714,0.587201,0.838064,1.2,1.2,1.0,6.0,29.2,0.204658
logistic regression,0.78209,0.731143,0.530165,0.612464,0.840573,1.2,1.2,1.0,4.6,29.2,0.157323
linear svm (L),0.462687,0.399772,0.897431,0.53234,0.464363,0.4,1.2,0.4,0.8,29.2,0.030484
non-linear svm (L),0.702985,0.544513,0.746173,0.617383,0.825117,0.0,1.2,0.0,0.0,29.2,0.0
logistic regression (L),0.777612,0.756847,0.481489,0.585173,0.825971,1.2,1.2,1.0,5.8,29.2,0.200582


,accuracy,precision,recall,f1,auc,n_violation,n_rule,violation_rate,n_violation (instance),n_evaluation (instance),violation_rate (instance)
RuleFit Classifier (disc),0.783582,0.742431,0.521271,0.610252,0.815651,0.8,1.2,0.8,2.0,29.2,0.069582
tree generator (disc),0.741791,0.627843,0.557706,0.582608,0.772322,1.2,1.2,1.0,5.2,29.2,0.18568
RuleFit Classifier (conti),0.732836,0.601979,0.553471,0.571908,0.795116,1.2,1.2,1.0,6.2,29.2,0.213997
tree generator (conti),0.78209,0.715158,0.568106,0.62746,0.8326,1.0,1.2,0.9,5.2,29.2,0.18141
linear svm,0.779104,0.711436,0.551175,0.618693,0.837805,0.8,1.2,0.8,2.6,29.2,0.090093
non-linear svm,0.780597,0.75907,0.479714,0.587201,0.838064,1.2,1.2,1.0,6.0,29.2,0.204658
logistic regression,0.78209,0.731143,0.530165,0.612464,0.840573,1.2,1.2,1.0,4.6,29.2,0.157323
linear svm (L),0.434328,0.386968,0.927563,0.524839,0.607319,0.2,1.2,0.2,1.2,29.2,0.046154
non-linear svm (L),0.704478,0.53671,0.768934,0.624831,0.8266,0.0,1.2,0.0,0.0,29.2,0.0
logistic regression (L),0.765672,0.709353,0.49182,0.576199,0.825963,1.2,1.2,1.0,5.2,29.2,0.178531


## 実験結果の保存（クロスバリデーション）

In [18]:
import os

for version, (dfs, info) in enumerate(zip(result_dfs, infos)):
    
    for nth_fold, df in enumerate(dfs):

        df['nth_fold'] = nth_fold
        df['n_splits'] = 5
        df['C1'] = 10
        df['C2'] = 10
        df['n_unsupervised'] = 15
        df['rule_thr'] = info

        # フォルダーを作成する
        folder_path = os.path.join(project_dir_path, f'experiment_result/results_organized/experiment_15/fold_{nth_fold}')
        os.makedirs(folder_path, exist_ok=True)

        file_path = f'{folder_path}/ver_{version}.csv'
        print(file_path)

        df.to_csv(file_path)


/Users/keisukeonoue/ws/lukasiewicz_2/experiment_result/results_organized/experiment_15/fold_0/ver_0.csv
/Users/keisukeonoue/ws/lukasiewicz_2/experiment_result/results_organized/experiment_15/fold_1/ver_0.csv
/Users/keisukeonoue/ws/lukasiewicz_2/experiment_result/results_organized/experiment_15/fold_2/ver_0.csv
/Users/keisukeonoue/ws/lukasiewicz_2/experiment_result/results_organized/experiment_15/fold_3/ver_0.csv
/Users/keisukeonoue/ws/lukasiewicz_2/experiment_result/results_organized/experiment_15/fold_4/ver_0.csv
/Users/keisukeonoue/ws/lukasiewicz_2/experiment_result/results_organized/experiment_15/fold_0/ver_1.csv
/Users/keisukeonoue/ws/lukasiewicz_2/experiment_result/results_organized/experiment_15/fold_1/ver_1.csv
/Users/keisukeonoue/ws/lukasiewicz_2/experiment_result/results_organized/experiment_15/fold_2/ver_1.csv
/Users/keisukeonoue/ws/lukasiewicz_2/experiment_result/results_organized/experiment_15/fold_3/ver_1.csv
/Users/keisukeonoue/ws/lukasiewicz_2/experiment_result/results_o

## 実験結果の保存（クロスバリデーション，平均と標準偏差）

In [4]:
for version, (info, df) in enumerate(zip(infos, dfs_mean)):
    df['C1'] = 10
    df['C2'] = 10
    df['n_unsupervised'] = 15
    df['rule_thr'] = info

    file_path = os.path.join(project_dir_path, f'experiment_result/results_organized/experiment_15/ver_{version}_mean.csv')
    print(file_path)
    df.to_csv(file_path)

for version, (info, df) in enumerate(zip(infos, dfs_std)):
    df['C1'] = 10
    df['C2'] = 10
    df['n_unsupervised'] = 15
    df['rule_thr'] = info

    file_path = os.path.join(project_dir_path, f'experiment_result/results_organized/experiment_15/ver_{version}_std.csv')
    print(file_path)

    df.to_csv(file_path)



/Users/keisukeonoue/ws/lukasiewicz_2/experiment_result/results_organized/experiment_15/ver_0_mean.csv
/Users/keisukeonoue/ws/lukasiewicz_2/experiment_result/results_organized/experiment_15/ver_1_mean.csv
/Users/keisukeonoue/ws/lukasiewicz_2/experiment_result/results_organized/experiment_15/ver_2_mean.csv
/Users/keisukeonoue/ws/lukasiewicz_2/experiment_result/results_organized/experiment_15/ver_3_mean.csv
/Users/keisukeonoue/ws/lukasiewicz_2/experiment_result/results_organized/experiment_15/ver_4_mean.csv
/Users/keisukeonoue/ws/lukasiewicz_2/experiment_result/results_organized/experiment_15/ver_5_mean.csv
/Users/keisukeonoue/ws/lukasiewicz_2/experiment_result/results_organized/experiment_15/ver_6_mean.csv
/Users/keisukeonoue/ws/lukasiewicz_2/experiment_result/results_organized/experiment_15/ver_7_mean.csv
/Users/keisukeonoue/ws/lukasiewicz_2/experiment_result/results_organized/experiment_15/ver_0_std.csv
/Users/keisukeonoue/ws/lukasiewicz_2/experiment_result/results_organized/experiment

## ルールの整形と保存（csv）

In [46]:
import ast

for nth_fold in range(5):
    for version in range(len(infos)):
        dir_path = os.path.join(project_dir_path, f"experiment_result/tmp/version_15{version + 1}")
        file_path_1 = f"{dir_path}/rules/rules_{nth_fold}_original.csv"
        file_path_3 = f"{dir_path}/result.json"

        with open(file_path_3, 'r') as f:
            result = json.load(f)
        rule_result = result['result'][f'fold_{nth_fold}']['rule_based_prediction']['violation_detail']
        df_rule_result = pd.DataFrame(rule_result).T

        df = pd.read_csv(file_path_1, index_col=0)

        tmp = []
        for _, row in df.iterrows():

            row['antecedents_length'] = len(eval(row['antecedents']))
            row['antecedents'] = " ⊗ ".join(eval(row['antecedents']))
            conseq_tmp = eval(row['consequents'])
            # for item in conseq_tmp:
            #     if row['lift'] >= 1:
            #         row['consequents'] = item
            #     else:
            #         row['consequents'] = "¬ " + item 
            #     tmp.append(row.copy())
            for item in conseq_tmp:
                row['consequents'] = item
                tmp.append(row.copy())

        df = pd.concat(tmp, axis=1, ignore_index=True).T
        df = df[['antecedents', 'consequents', 'support', 'confidence', 'lift', 'antecedents_length']]
        df['n_violation'] = None
        df['n_violation (instance)'] = None
        df['n_evaluation (instance)'] = None
        
        for i, row in df[df['consequents'] == 'Outcome'].iterrows():
            row['n_violation (instance)'] = df_rule_result[1].to_list()

        print()
        print()
        print(len(df[df['consequents'] == 'Outcome']))
        print(len(df_rule_result))
        print()
        print()

        df.loc[df['consequents'] == 'Outcome', 'n_violation (instance)'] = df_rule_result[1].to_list()
        df.loc[df['consequents'] == 'Outcome', 'n_evaluation (instance)'] = df_rule_result[0].to_list()
        df.loc[df['consequents'] == 'Outcome', 'n_violation'] = df['n_violation (instance)'].apply(lambda x: 0 if x == 0 else 1)

        df['consequents'] = df['consequents'].where(df['lift'] >= 1, "¬ " + item)

   
        file_path = os.path.join(project_dir_path, f'experiment_result/results_organized/experiment_15/fold_{nth_fold}/rules_{version}.csv')
        df.to_csv(file_path)




48
48




48
48




32
32




15
15




3
3




1
1




1
1




1
1




44
44




44
44




28
28




13
13




4
4




2
2




1
1




1
1




39
39




39
39




25
25




12
12




3
3




1
1




1
1




1
1




51
51




51
51




39
39




21
21




8
8




4
4




2
2




2
2




48
48




48
48




33
33




15
15




6
6




1
1




1
1




1
1


